In [3]:
# ----- 環境変数読み込み -----
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# ----- OpenAI準備 -----
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Hi there! How can I assist you today?"
}


In [15]:
# ----- swagger.yaml読み込み -----
import yaml
import json

with open('/Users/rhirabay/dev/github.com/rhirabay/my-projects/python/openai/swagger.yaml') as file:
    swagger_yaml = yaml.safe_load(file)
    # print(swagger_yaml['info']['title'])

with open('/Users/rhirabay/dev/github.com/rhirabay/my-projects/python/openai/swagger.json', 'w', encoding='UTF-8') as file:
    swagger_json = json.dumps(swagger_yaml, indent=2)
    file.writelines(swagger_json)


In [25]:
# ----- lang chain -----
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

# json読み込み
# loader = JSONLoader(
#     file_path='/Users/rhirabay/dev/github.com/rhirabay/my-projects/python/openai/swagger.json',
#     jq_schema = '.',
# )
# data = loader.load()

loader = TextLoader('/Users/rhirabay/dev/github.com/rhirabay/my-projects/python/openai/swagger.json')
data = loader.load()
print(data)

[Document(page_content='{\n  "openapi": "3.0.0",\n  "info": {\n    "title": "\\u30e6\\u30fc\\u30b6API",\n    "description": "\\u30e6\\u30fc\\u30b6\\u306b\\u95a2\\u3059\\u308b\\u64cd\\u4f5c\\u3092\\u96c6\\u7d04\\u3057\\u305fAPI",\n    "version": "0.0.1"\n  },\n  "servers": [\n    {\n      "url": "https://api.example.com/v1",\n      "description": "\\u672c\\u756a\\u30b5\\u30fc\\u30d0"\n    }\n  ],\n  "paths": {\n    "/users": {\n      "post": {\n        "summary": "\\u30e6\\u30fc\\u30b6\\u767b\\u9332",\n        "description": "\\u30e6\\u30fc\\u30b6\\u3092\\u767b\\u9332\\u3059\\u308b",\n        "requestBody": {\n          "content": {\n            "application/json": {\n              "schema": {\n                "type": "object",\n                "properties": {\n                  "username": {\n                    "type": "string",\n                    "description": "\\u30e6\\u30fc\\u30b6\\u540d"\n                  }\n                }\n              }\n            }\n          }\n     

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(data, embedding=embeddings, persist_directory=".")
vectorstore.persist()

# 学習させる
pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(),chain_type='refine' ,return_source_documents=True)

print('ok')

ok


In [31]:
# ----- curlコマンドを生成させてみる -----
query = "please show me curl command to create user with name 'hirabay'."
chat_history = []

result = pdf_qa({"question": query, "chat_history": chat_history})

result["answer"].replace("\\n", "\n")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'Based on the provided context, the refined curl command to create a user with the name \'hirabay\' would be:\n\n```\ncurl -X POST -H "Content-Type: application/json" -d \'{"username": "hirabay"}\' https://api.example.com/v1/users\n```\n\nPlease note that you need to replace `https://api.example.com/v1` with the actual URL of the server where the API is hosted. This command will send a POST request to the `/users` endpoint of the API, with the request body containing the username \'hirabay\'.'

In [ ]:
# ----- API定義について聞いてみる -----
query = "please show me curl command to create user with name 'hirabay'."
chat_history = []

result = pdf_qa({"question": query, "chat_history": chat_history})

result["answer"]